In [1]:
'''
Preprocess script for SNPs, mostly covering the translation part
synonomous vs non-synonomous
'''
import pandas as pd, numpy as np, re, sys, json, requests, re
from pathlib import Path
from multiprocessing import Pool
from collections import Counter
from subprocess import run, PIPE

in_path = Path('/d/data/plasmo/nat_out/v3')
in_file = in_path / 'snps_by_genes.tsv'
out_path = in_path / 'frames.tsv'
genes_file = in_path / 'filtered_genes.txt'
ref_file = in_path / '3d7.fasta'

In [2]:
#This is an adapted function to fetch the strandness from the gene location
def getLocation(row):
    col_names = ['id', 'loc', 'name', 'symbol']
    p = '[(]([+-])[)]'
    
    loc = row[col_names[1]]
    position = re.search(p, loc) #position is the search object
    strand = position.group(1).strip()
        
    return pd.Series([row[col_names[0]], strand])

genes_df = pd.read_csv(genes_file, sep='\t')
strand_df = genes_df.apply(getLocation, axis=1) #modified version containing decomposed position
strand_df.columns = ['id', 'strand']
strand_df.set_index('id', inplace=True)


In [3]:
#loading the reference sequence to eventually get codons
p = '(?s)[>](.+?) [|].+?\n(.+?)(?=[>]|$)'
with open(ref_file, 'r') as input:
    data = input.read()
    data = re.findall(p, data)
    ref_dict = {}
    for i, e in data:
        ref_dict[i] = e.replace('\n', '')
    data = None

In [4]:
#loading the actual SNPs
in_df = pd.read_csv(in_file, sep='\t', index_col = [0,1])


In [5]:
# with open(codons_file, 'r') as input:
#     codons = json.loads(input.read())
    
by_genes = in_df.groupby('CHROM') #each iteration yields (name, df)

In [6]:
def rev_comp(s):
    z_dict = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return ''.join([z_dict[l] for l in s][::-1])

def blast(seq, id, rev, offset):
#returns the predicted frame for translation
#  values to use in caller  
#     seq = seq.upper()
#     snp_ind = 20
#     offset = 15
#     rev = False
    
    def parseAlignment(text): #takes only one line
        try:
            indices = {
                'id': 1,
                'identity': 2,
                'self_start': 6,
                'self_end': 7,
                'sub_start': 8,
                'sub_end': 9
            }
            elements = text.strip('\n').split('\t')            
            id = elements[indices['id']]
            sub_start = int(elements[indices['sub_start']])
            sub_end = int(elements[indices['sub_end']])
            self_start = int(elements[indices['self_start']])
            self_end = int(elements[indices['self_end']])
            identity = float(elements[indices['identity']])
#             if identity < 100.:
#                 print('{1} percent match at gene {0}'.format(id, identity))
        except Exception  as e:
#             print(text)
#             print(e)
            raise Exception

        return {'id': id, 'sub_start': sub_start, 'sub_end': sub_end, 'self_start': self_start, 'self_end': self_end}


    predicted_f = None
    path_to_db = Path('/d/data/plasmo/nat_out/transcripts')
    command = 'echo {seq} | blastn -db {db} -task blastn-short -dust no -soft_masking false -outfmt 6'.format(seq = seq, db = path_to_db)
    record = run(command, shell=True, encoding='utf-8', stdout=PIPE).stdout
#     print(record)

    for line in record.split('\n'):
        data = parseAlignment(line)
        if re.search(id, data['id']):
            if not ( data['self_start'] > offset or data['self_end'] < offset):
                if rev:
                    predicted_f = (int(data['sub_end']) + (data['self_end'] - offset * 2 - 1)) % 3
                else:
                    predicted_f = (int(data['sub_start']) - (data['self_start'] - 1)) % 3
                break


    return int(predicted_f)

In [7]:
#function for assigning score to each element of each row
def findFrame(row):               
    #the name of the chromosome should be id:chromosome
    full_id = row.name[0].split(':')
    id = full_id[0]
    chr = full_id[1]
    pos = int(row.name[1] - 1) #positions are 1 indexed
    rev = strand_df['strand'].loc[id] == '-'
    offset = 15

    try:
        ref_chunk = ref_dict[chr][pos - offset : pos + offset + 1]
    except:
        print('ref chunk out of bound at {0} {1}'.format(row.name[0], pos))
        return None
    
#     print(ref_chunk, id, rev)
    try:
        frame = blast(ref_chunk, id, rev, offset)
    except Exception as e:
        print(ref_chunk, id, rev)
#         print(e)
        frame = None
    return frame
    
def worker(args):
    #hard coded args
    name = args[0]
    snps_df = args[1]
#     print(list(snps_df.index.get_level_values('POS')))
#     sys.exit(0)
    
    frames = snps_df.apply(findFrame, axis = 1)
    return frames
with Pool(6) as pool:
    res_dfs = pool.map(worker, list(by_genes))
    
res = pd.concat(res_dfs)
res.columns = ['frames']

TTATTTTATTCTATTCTTTTTTATATGTCAT PF3D7_0102200 False
TTTATATATTATAATACCGTTTTTAATAATA PF3D7_0102200 False
ATATATTATAATACCGTTTTTAATAATATAT PF3D7_0102200 False
TACGAAAAAAAAAAATAAATAAATAAATAAT PF3D7_0102500 True
TTTTTTTTTTTTTTTGTTTTTTAGTTTTTTC PF3D7_0102500 True
ATTTCTTCATCCTTTTTTTCTTACAAATTCT PF3D7_0102500 True
TATATGTTGTATGTATTTTTATGTATACATA PF3D7_0311400 False
TAATGTATATATAATGTATATATAATGTATA PF3D7_0311400 False
AAAAAATTTAATATATATATATGTATTTTTA PF3D7_0102600 True
AATAAATAAATATATGAACGTACATAGGTTT PF3D7_0416400 False
AAAAAAGATAATATTTGCATGTATATATATT PF3D7_0416400 False
TATATTTCAATGTGTGTGCAACTAGCCATTT PF3D7_0416400 False
TGTTATATAACAATCTATTAAAAATATATAT PF3D7_0514600 True
TATAATAACATATACTTAAATTCATATATTC PF3D7_0514600 True
GCTATCAAAAAAAAAAAAAAAAAAATAAAAT PF3D7_0416400 False
AAAATATTTGTTCATGTTTCCTTGTTACCAT PF3D7_0514600 True
TATATATATATGTATGTATGTATGTATGTAT PF3D7_0514600 True
AATAAAATATATATATATATATATATATATA PF3D7_0311600 False
CATATTTATGCATATTTTTTTTTATTATGTA PF3D7_0311600 False
TGGTAAGTTAAAAAGGGGGT

TATATATATATATAATTATACTTCGTTTAGT PF3D7_0418500 True
GTTCATAAATATATAAACATTTTTTATATAC PF3D7_0418500 True
ATATTATCATATTTTAATATATATACATATT PF3D7_0316600 True
ATTTTTTTTTTTTTTTTTATTTTTTTTTTTT PF3D7_0418500 True
AATTGTGCACAAAAATTAAATAAAGTATAAG PF3D7_0418500 True
ATGGAATGAATGATGAAATATATGATGAAAT PF3D7_0316600 True
AATGAATGATGAAATATATGATGAAATATAT PF3D7_0316600 True
TATTATAAATAATAAAGAATATATGTATATT PF3D7_0418500 True
ATGAATGATGAAATATATGATGAAATATATG PF3D7_0316600 True
TATGATGGAATAAATGATAAAATATATGATA PF3D7_0316600 True
AATGATAAAATATATGATAAAATATATAACG PF3D7_0316600 True
TGATAAAATATATAACGAAATATATGATAAA PF3D7_0316600 True
TCATATACAATGTATCAATAAAAAATAAATA PF3D7_0316600 True
ATGCATACACTTTATAATATAAAAGTCAAAC PF3D7_0316600 True
AAAAAAAATACATATATATAAATAAATATAA PF3D7_0316700 False
TTTATAACATATTGTGTATATATGTAAACAC PF3D7_0316700 False
TAACATATTGTGTATATATGTAAACACTATT PF3D7_0316700 False
ATAAACCAATTATAAACATATACACATATAT PF3D7_0316700 False
TAAACATATACCTATATAATTGTGTATATTT PF3D7_0316700 False
ACATATATCCCCTTTTACATTTTGAT

GTACTTACGTATGTAGTTATGCATTTTTTTA PF3D7_0518700 True
ATGTGTACATTTTTTTATGTATATATTAATT PF3D7_0518700 True
TATATATATATTTATTTATTTATTTATTTAT PF3D7_0518700 True
AAAATAATTGTTGGGCATGTATAAATGTATA PF3D7_0613800 False
TATGTTTAATATGTTTAATATATATATAATA PF3D7_0613800 False
ATAAATAAATAAATATATATATATATATATA PF3D7_0107000 False
AATTTTTTTTTTATATCTTTACAATATATAA PF3D7_0107000 False
TTTTTTTTTTATATCTTTACAATATATAACA PF3D7_0107000 False
TTTATTAAATTAAATACATCAAATTGTATCT PF3D7_0107000 False
ATATATATATATATAAATATATATATATTAC PF3D7_0320000 False
AAATACATCAAATTGTATCTTTTGTTTGTTT PF3D7_0107000 False
AGAAAAAAAAAAAAATAATAATAATAATAAA PF3D7_0215900 True
TATATATATATATATGTATATTTTATTTATT PF3D7_0320000 False
ATAATATAATGAATCTTATGTAAGCATAAGT PF3D7_0215900 True
TTTTTTTTTTTTTTTTTTTTTTCATGTTAGC PF3D7_0107000 False
ATTAACATTTAAAAATTTTTTTTTATTTTGT PF3D7_0320000 False
TAAGTATATATATAGATATATATTTTTTTTA PF3D7_0215900 True
TGTTGATATATATATATACATATATATATAT PF3D7_0320000 False
ATATATATAAAATATTTGTGATGTATCAAAA PF3D7_0107500 True
TATTTATTTATTTATTTAC

TATATATATATATATATATATATATATATAA PF3D7_0219500 True
TTATTTTAATAAATTATGTGGCGTTTATAAA PF3D7_0219500 True
TATATATATATTTATTTATTTATTTATTTAT PF3D7_0615800 True
TATATATATATATATTTATTTATTTATTTAT PF3D7_0301200 True
ACGTAAGAATATTGAGGCACTATGTGAATAC PF3D7_0522400 False
TATATTTATTTATTTTTAAATTTTTGTGTGT PF3D7_0301200 True
AAATAAATATTCAAACATGTATTTAATATAT PF3D7_0110400 True
ATAAATATTCAAACATGTATTTAATATATAT PF3D7_0110400 True
AAAAATATATATATATATATATATATATATA PF3D7_0616200 False
ATATATGTGTGTGTGTGGTATCCATTTTATT PF3D7_0616200 False
AATACAAATATTATATTGAAATGTTTATACG PF3D7_0422800 False
CATTTATACCTATTTGTACACATTTTAGGCC PF3D7_0422800 False
AAATATAAATATAAAGAAAAAAGAAGCATAA PF3D7_0110600 False
TATATTGTATATATAACATATATATATATAT PF3D7_0422800 False
AATATATATATCAAATTTTTCATTGTATAAT PF3D7_0422800 False
TAAAAGGAAGATATACAATTATAAATATAAT PF3D7_0110600 False
TCAAATTTTTCATTGTATAATTTATTACAAT PF3D7_0422800 False
GGAAGATATACAATTATAAATATAATATATT PF3D7_0110600 False
AAAAAAAAAAAAAAAAAAATTGTTATTATGA PF3D7_0422800 False
TAATACAATGTAAATAGAA

AAAAAAAAAAATATATATATAAATACATATA PF3D7_0404300 True
TATACGTATAAATTTTTTATTATTTTTTTTT PF3D7_0404300 True
ATATATATATATATATATATATATATACATT PF3D7_0404300 True
CAAAAAAAAAAAAAATAAAATAATAATATAT PF3D7_0404300 True
AAAAATAAAATAATAATATATTTTATATACA PF3D7_0404300 True
CTTTCGTTATCGCTGAAAAAAAAAAAAAAAA PF3D7_0404300 True
TATATATTATATATAAATAAATAAATATAAA PF3D7_0404300 True
ATGAAAAAAAAAAAAAAATAATAAAAAAAAT PF3D7_0304000 True
AAAAAAAAATAATAAAAAAAATAATTAAATA PF3D7_0304000 True
AAAAATAATAAAAAAAATAATTAAATATATA PF3D7_0304000 True
TAATAAAAAAAATAATTAAATATATACAAGA PF3D7_0304000 True
ATTAAATATATACAAGATTGCTATGCATATA PF3D7_0304000 True
AAAAATAAAAAATAAATATAATATGTAAACA PF3D7_0304000 True
ATATATATATATATAAATATATATGTGTGTG PF3D7_0304000 True
TAAAATGAGGAATGCGTCAAAAAGAAAATGC PF3D7_0304000 True
TTATTATACCGGGTAAGATATGAACACATAA PF3D7_0113800 False
CCGGGTAAGATATGAACACATAAGATATAGC PF3D7_0113800 False
AAACAATAAAAAAAAAATATATTTATATATA PF3D7_0621350 True
ATGAACACATAAGATATAGCAATTAATATAA PF3D7_0113800 False
ACTACCTGGAAAAAGTGTATTATAAAAG

...

TTATATATCTAAGCAAGAACAAATTTAATAT PF3D7_1465300 False
AAAATATAAATTAAAATTTTTTTTTTTTCTT PF3D7_1345100 True
AAATATAAATTAAAATTTTTTTTTTTTCTTT PF3D7_1345100 True
AGTTAAAATAATGAACAGGGTATATATTTTT PF3D7_1345100 True
ATTTTATCCATGTTTGTAGGCATGTAAAAAA PF3D7_1428500 False
AATATAAATATAAATGTAAATATATATATAT PF3D7_1465300 False
AAAAAAATAATAAGATAAAATAAAAAAATAT PF3D7_1465300 False
TAAAAATGGAAAAAAATTATTCAATATATAA PF3D7_1465300 False
AAATATACATACATAGATATATACATACATA PF3D7_1346000 True
ATATATATATATATACATATATATGTTTTTA PF3D7_1361100 False
AAAATAAATACATACATACATACTTACATAT PF3D7_1444300 False
TAAATACATACATACATACTTACATATATAT PF3D7_1444300 False
ATAAATATAATTTTATCTACCATTTGTTATA PF3D7_1444300 False
AGGCATATCTACAAAAAAAGTAAATGATATT PF3D7_1361400 True
TTGAGTTTTTATAGGCCACCTTATATATATA PF3D7_1444300 False
TACAAAAAAAGTAAATGATATTATACAAGAG PF3D7_1361400 True
AATATATATATATATATATTAATTATTTTAT PF3D7_1444300 False
TTTTATATTTATAGAGATATAACAAAAAGAT PF3D7_1361400 True
AAAAAGTTATATATATATATATATATATATA PF3D7_1444300 False
CCTATTGAACATTATGGAG

AAATTCAAAATAAAATATATATATATATATA PF3D7_1431900 False
TATATATATATATATGTATGTATGTACTCAT PF3D7_1366900 False
ATATATTTTCATATATATTTTCATATATATT PF3D7_1366900 False
TCATATATATTTACATATATTTATCCATTCA PF3D7_1366900 False
AAAAGAAAAAAATTCAAATGGAAGTATCATA PF3D7_1432100 False
AAATAAATATACATATATATATATATATATA PF3D7_1432100 False
ATAAAATAAATATATAGATATTCATTAATGT PF3D7_1432100 False
TAAAATATTATAAGTCTATATTTTATCTTTT PF3D7_1432100 False
TTTTATCTTTTTTGAACTTACATAAATATAT PF3D7_1432100 False
ACATATTTTTAACAATCTTTTAGCCTGGTTT PF3D7_1432100 False
TATATATATATATATTTATATATTTATATTT PF3D7_1432100 False
TTTTACTTATGAACATATACACTTATGTAGA PF3D7_1432100 False
TATATATATTTATATTTATATTTATATTTAT PF3D7_1432100 False
TATATATATATATATTTATTTAATTATATAT PF3D7_1432100 False
ATATATATATATATACATTTATGTGTTTGTT PF3D7_1367000 True
TTATTTTATTTTTTATTTTACTTGTTTTTCT PF3D7_1367000 True
GAACACATATATATATATATATATATATATA PF3D7_1367000 True
TGTATGTAATAATTTGTGTTAATTTTTATTC PF3D7_1432300 True
ATATTAATATACATCCATATAAAAAATATGT PF3D7_1367000 True
AATATGTACATTTCTTT

TATATATATATATATATGTGTTACTCTTTAT PF3D7_1434000 True
TATATCCTTACCACTGTAAAAAACAAAGAAA PF3D7_1474700 True
TATATATATATTTATTTATTTATTTATTTAT PF3D7_1474700 True
GTGTTTACTTATTTACATATACATATATATA PF3D7_1452600 False
TATGAAATGTACATATATCTTTATTTTCATT PF3D7_1452600 False
TAATTGTTTTGTTTTGTTTTTTCTTATATTT PF3D7_1452600 False
AATATATACATATATACACATTTAAATAAAT PF3D7_1452600 False
GACAGATATAAGAAAAAATTAAATTTTTAAA PF3D7_1416900 True
CATCAGGTAATAGTCTATATATAAAGTAACA PF3D7_1452600 False
TTAAGGCTTATAATTCAATGTAAAAATTATA PF3D7_1416900 True
AGATAGTATTATTACGTATATTCAAATAAAA PF3D7_1416900 True
ATTCAAATAAAATATGCATATATATATATAT PF3D7_1416900 True
AATCTACAAAATTGTTATAAAAAAATAAAAG PF3D7_1416900 True
TAGGAAGGTACAACCAGATAAAAAATATAAT PF3D7_1452700 False
TTAAACCATTAAAAAAATAATATTCCCTATT PF3D7_1416900 True
TGTCAAAAAAAAAAAAAAAAAAAAAAAAAAT PF3D7_1452700 False
TATATATATGTGTATGAATTAATGAAAGCAC PF3D7_1474800 False
TATATATTTATTTATTTTATATATTTCCTTA PF3D7_1452700 False
TATTATAACACAGAATATTTTGCATAATGTA PF3D7_1452700 False
GTGGTAAGTTATTTTATGAAA

ref chunk out of bound at PF3D7_API04300:Pf3D7_API_v3 17667
ref chunk out of bound at PF3D7_API04300:Pf3D7_API_v3 17702
ref chunk out of bound at PF3D7_API04300:Pf3D7_API_v3 17711
ref chunk out of bound at PF3D7_API04300:Pf3D7_API_v3 17713
ref chunk out of bound at PF3D7_API04300:Pf3D7_API_v3 17714
ref chunk out of bound at PF3D7_API04300:Pf3D7_API_v3 17726
ref chunk out of bound at PF3D7_API04300:Pf3D7_API_v3 17756
ref chunk out of bound at PF3D7_API04300:Pf3D7_API_v3 17760
ref chunk out of bound at PF3D7_API04300:Pf3D7_API_v3 17798
ref chunk out of bound at PF3D7_API04300:Pf3D7_API_v3 17800
ref chunk out of bound at PF3D7_API04300:Pf3D7_API_v3 17804
ref chunk out of bound at PF3D7_API04300:Pf3D7_API_v3 17811
ref chunk out of bound at PF3D7_API04300:Pf3D7_API_v3 17879
ref chunk out of bound at PF3D7_API04300:Pf3D7_API_v3 17898
ref chunk out of bound at PF3D7_API04300:Pf3D7_API_v3 17927
ref chunk out of bound at PF3D7_API04300:Pf3D7_API_v3 17945
ref chunk out of bound at PF3D7_API04300

AAAAAAAAAAAAAAAGAAATATATTATATAA PF3D7_1440300 False
CATATGTGTATATACCAATTTCTCATATTTT PF3D7_1457000 True
TATTATATAATTGTACGTATAAATGAACGTA PF3D7_1440300 False
ATAAATATATATATATATATATATATATATA PF3D7_1440300 False
AGGAGTTTATTTTTCTTGTAAATACCTTTAT PF3D7_1457000 True
TTCAGCAAATAAATGTATTATATATAAATAT PF3D7_1457000 True
TCAGCAAATAAATGTATTATATATAAATATT PF3D7_1457000 True
CAGGTAATAATTTATCAATATATTTCATTGA PF3D7_1440300 False
ATTAATAAATTTTTTTTTTATTTTTTCATTT PF3D7_1457000 True
TTATTTATTTATTTATTTATGAACTTCTTTT PF3D7_1440300 False
ATTATAATGTGGGAATGTGTAAAAAGCTATG PF3D7_1457200 True
TATAATGTGGGAATGTGTAAAAAGCTATGTA PF3D7_1457200 True
ATAATGTGGGAATGTGTAAAAAGCTATGTAA PF3D7_1457200 True
AAACATAAAATTTTACACTTATTATATATTT PF3D7_1457200 True
TAATATGCATTAAATGTATGCTTATATATTT PF3D7_1457200 True
TCCATTTCCTTATCACCAATATAAAAAGTAT PF3D7_1460400 False
TAATTATTATAAAACTTCATTTGTTATATAA PF3D7_1460400 False
TGTAATATGAATAATATTTTGCCAATTTTGT PF3D7_1460400 False
ACACACACACACATATATATATATATATATA PF3D7_1460400 False
TTTTTAATACATTATAAAATGG

In [8]:
# res.T.to_csv(out_path, sep='\t')
res.to_csv(out_path, sep='\t')

/home/javi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
